In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Sun May 15 15:02:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   44C    P8    39W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 29%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    # db_test1 = db.test_1
    # db_test2 = db.test_2
    # db_rm_us = db.rm_patent_us
    # db_rm_cn = db.rm_patent_cn
    # db_rm_de = db.rm_patent_de
    
    db_cn_5 = db.w_patent_cn_5
    db_cn_10 = db.w_patent_cn_10
    db_us_5 = db.w_patent_us_5
    db_us_10 = db.w_patent_us_10
    db_de_5 = db.w_patent_de_5
    db_de_10 = db.w_patent_de_10
    
except errors.ConnectionFailure as err:
    print(err)

In [6]:
db_cn_5

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'w_patent_cn_5')

# CPC計算

## 前五年

### DE

In [7]:
data_assignee = db_de_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [8]:
len(ls_company)

63

In [9]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_de_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_de = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_de

,company,country,count
0,Robert Bosch GmbH,DE,17
1,Robert Bosch GmbH,CN,9
2,Robert Bosch GmbH,US,7
3,Robert Bosch GmbH,WO,6
4,Robert Bosch GmbH,EP,5
...,...,...,...
271,MOOREDOLL Inc,GB,1
272,MOOREDOLL Inc,US,1
273,MOOREDOLL Inc,JP,1
274,MOOREDOLL Inc,CN,1


In [10]:
df_family_de.shape

(276, 3)

In [11]:
# 轉成company-country 的總數表格
table_de = df_family_de.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

country,company,AT,BR,CA,CH,CN,DE,DK,EP,ES,...,LT,MX,PL,PT,RS,RU,SG,TW,US,WO
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,Accuragen Inc,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0
2,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,Apple Inc,0.0,0.0,0.0,0.0,1.0,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,12.0,12.0
4,Armstrong DLW GmbH,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
final_table_de.shape

(62, 26)

In [13]:
final_table_de.to_csv('w_company_country_de_5.csv',index=0)

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_de_5

In [15]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"country":final_table_de.columns.tolist()[1:]})

In [16]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### CN

In [17]:
data_assignee = db_cn_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [18]:
len(ls_company)

2077

In [19]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_cn_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_cn = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_cn.head()

,company,country,count
0,Jiangsu IoT Research and Development Center,CN,364
1,Jiangsu IoT Research and Development Center,WO,8
2,Nanjing IoT Sensor Technology Co Ltd,CN,204
3,Nanjing IoT Sensor Technology Co Ltd,WO,1
4,Nanjing IoT Sensor Technology Co Ltd,US,1


In [20]:
df_family_cn.shape

(5302, 3)

In [21]:
# 轉成company-country 的總數表格
table_cn = df_family_cn.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

country,company,AP,AR,AU,BR,CA,CL,CN,CO,CR,...,SI,SM,TN,TR,TW,UA,US,UY,WO,ZA
0,001 Engineering Co ltdqingdao Construction Group,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1st Engineering Co Ltd of MBEC,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2nd Engineering Co Ltd of MBEC,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
4,3M innovation intellectual property Co.,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [22]:
final_table_cn.shape

(2000, 62)

In [23]:
final_table_cn.to_csv('w_company_country_cn_5.csv',index=0)

In [24]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_cn_5

In [25]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"country":final_table_cn.columns.tolist()[1:]})

In [26]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### US

In [27]:
data_assignee = db_us_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [28]:
len(ls_company)

705

In [29]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    try:
        data_fam = db_us_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    except:
        print(c)
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_us = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_us.head()

,company,country,count
0,Samsung Electronics Co Ltd,US,383
1,Samsung Electronics Co Ltd,KR,317
2,Samsung Electronics Co Ltd,WO,155
3,Samsung Electronics Co Ltd,CN,145
4,Samsung Electronics Co Ltd,EP,141


In [30]:
df_family_us.shape

(2251, 3)

In [31]:
# 轉成company-country 的總數表格
table_us = df_family_us.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

country,company,AR,AU,BR,CA,CN,DE,DK,EA,EC,...,RS,RU,SA,SE,SG,TW,UA,US,WO,ZA
0,3d-Oxides,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,AFI Licensing LLC,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,AFIRMA CONSULTING & TECHNOLOGIES SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [32]:
final_table_us.shape

(700, 40)

In [33]:
final_table_us.to_csv('w_company_country_us_5.csv',index=0)

In [34]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_us_5

In [35]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"country":final_table_us.columns.tolist()[1:]})

In [36]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

## 後五年

### DE

In [37]:
data_assignee = db_de_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [38]:
len(ls_company)

390

In [39]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_de_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_de = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_de

,company,country,count
0,Intel Corp,DE,449
1,Intel Corp,US,357
2,Intel Corp,CN,233
3,Intel Corp,WO,210
4,Intel Corp,KR,53
...,...,...,...
1374,Kitz Corp,WO,1
1375,Kitz Corp,US,1
1376,Kitz Corp,JP,1
1377,Kitz Corp,CN,1


In [40]:
df_family_de.shape

(1379, 3)

In [41]:
# 轉成company-country 的總數表格
table_de = df_family_de.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

country,company,AR,AT,AU,BE,BR,CA,CH,CL,CN,...,PH,PL,PT,RU,SE,SG,TW,US,WO,ZA
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
final_table_de.shape

(387, 39)

In [43]:
final_table_de.to_csv('w_company_country_de_10.csv',index=0)

In [44]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_de_10

In [45]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"country":final_table_de.columns.tolist()[1:]})

In [46]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### CN

In [47]:
data_assignee = db_cn_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [48]:
len(ls_company)

11480

In [49]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    data_fam = db_cn_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_cn = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_cn.head()

,company,country,count
0,Qualcomm Inc,CN,2543
1,Qualcomm Inc,WO,2515
2,Qualcomm Inc,US,2458
3,Qualcomm Inc,EP,2362
4,Qualcomm Inc,TW,914


In [50]:
df_family_cn.shape

(20363, 3)

In [51]:
# 轉成company-country 的總數表格
table_cn = df_family_cn.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

country,company,AR,AT,AU,BE,BG,BR,CA,CH,CL,...,SG,SI,TN,TR,TW,UA,US,UY,WO,ZA
0,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,10757616 Canada Co,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
3,10757616加拿大公司,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0
4,"11th Street Co., Ltd.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [52]:
final_table_cn.shape

(11059, 59)

In [53]:
final_table_cn.to_csv('w_company_country_cn_10.csv',index=0)

In [54]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_cn_10

In [55]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"country":final_table_cn.columns.tolist()[1:]})

In [56]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})

### US

In [57]:
data_assignee = db_us_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [58]:
len(ls_company)

4957

In [59]:
ls_c = []
ls_family = []
ls_count = []

for c in ls_company:
    try:
        data_fam = db_us_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$countrystatus"},
        {"$group" : {"_id" : "$countrystatus", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    except:
        print(c)
    

    for data in data_fam:
        #print(data)
        ls_c.append(c)
        ls_family.append(data["_id"])
        ls_count.append(data["count"])
        
df_family_us = pd.DataFrame((zip(ls_c, ls_family, ls_count)), columns = ['company', 'country', 'count'])
df_family_us.head()

,company,country,count
0,Qualcomm Inc,US,4649
1,Qualcomm Inc,WO,4189
2,Qualcomm Inc,CN,2487
3,Qualcomm Inc,EP,2208
4,Qualcomm Inc,TW,1131


In [60]:
df_family_us.shape

(14531, 3)

In [61]:
# 轉成company-country 的總數表格
table_us = df_family_us.pivot_table(index='company', values='count', columns='country').fillna(0).reset_index()
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

country,company,AP,AR,AT,AU,BE,BG,BR,CA,CH,...,SG,SI,TN,TR,TW,UA,US,UY,WO,ZA
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,12a Systems Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0


In [62]:
final_table_us.shape

(4919, 59)

In [63]:
final_table_us.to_csv('w_company_country_us_10.csv',index=0)

In [64]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.w_company_country_us_10

In [65]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"country":final_table_us.columns.tolist()[1:]})

In [66]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"country":k[1:]})